In [673]:
import requests
from bs4 import BeautifulSoup
import calendar
import re
import pandas as pd
import pickle
import string
from dateutil import parser
import time
import sys

In [597]:
prefix = 'https://www.ted.com/talks?page='

In [689]:
def get_soup(url):
    response = requests.get(url)
    status_c = response.status_code
    print (status_c)
    if status_c == 429:
        sys.stdout.flush()
        time.sleep(10)
    else:
        sys.stdout.flush()
        time.sleep(5)
        page = response.text
        return BeautifulSoup(page, "lxml")

In [550]:
def get_speaker_urls(soup):
    tags = soup.find_all(class_=" ga-link")[::2]
    anchors = []
    for item in tags:
        anchor = item.get('href')
        anchors.append(anchor)
    return anchors

In [566]:
speaker_urls = []
for i in range(1, 70):
    start_url = prefix + str(i)
    soup = get_soup(start_url)
    urls = get_speaker_urls(soup)
    speaker_urls.extend(urls)
    time.sleep(5)

In [586]:
speaker_urls[:2]

['/talks/wendy_troxel_why_school_should_start_later_for_teens',
 '/talks/ted_halstead_a_climate_solution_where_all_sides_can_win']

In [624]:
def get_speaker(soup):
    try:
        name = soup.find(class_="player-hero__speaker__content").text.replace(':', '')
        return name.strip()
    except:
        return ''

In [625]:
def get_title(soup):
    try:
        title = soup.find(class_="player-hero__title__content").text
        return title.strip()
    except:
        return ''

In [626]:
def get_seconds(soup):
    try:
        timestr = str(soup.find(class_="player-hero__meta").find_all('span')[0].text)
        ftr = [60,1]
        seconds = sum([a*b for a,b in zip(ftr, [int(i) for i in timestr.split(":")])])
        return seconds
    except:
        return ''

In [627]:
def get_date(soup):
    try:
        datelst = soup.find(class_="player-hero__meta").find_all('span')[1].text.split()
        month = datelst[1]
        month_dict = {v: k for k,v in enumerate(calendar.month_abbr)}
        year = int(datelst[2])
        return month_dict[month], year
    except:
        return ''

In [628]:
def view_count(soup):
    try:
        viewstr = soup.find("div", {"id": "sharing-count"}).find_all('span')[0].text
        views = int(viewstr.replace(',', ''))
        return views
    except:
        return ''

In [629]:
def get_transcript(soup):
    try:
        fragments = soup.find_all(class_="talk-transcript__fragment")
        text = []
        for i in fragments:
            t = i.get_text().strip().replace('\n',' ')
            if '(Laughter)' not in t and '(Applause)' not in t:
                text.append(t)
        return text
    except:
        return ''

In [687]:
def get_speaker_info(soup):
    try:
        info = soup.find(class_="talk-speaker__description").text.strip()
        return info
    except:
        return ''
        

In [1]:
# database = []

# for url in speaker_urls:
#     soup = get_soup('https://www.ted.com' + url)
#     db_dict = {}
#     db_dict['speaker'] = get_speaker(soup)
#     db_dict['url'] = url
#     db_dict['title'] = get_title(soup)
#     db_dict['info'] = get_speaker_info(soup)
#     db_dict['seconds'] = get_seconds(soup)
#     db_dict['date'] = list(get_date(soup))
#     db_dict['views'] = view_count(soup)
#     soup = get_soup('https://www.ted.com' + url + '/transcript?language=en')
#     db_dict['text'] = get_transcript(soup)
#     database.append(db_dict)